In [5]:
import os
import time

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

import urllib
import cv2

In [1]:
%load_ext tensorboard

In [2]:
logdir = "logs"

In [10]:
%tensorboard --logdir={logdir}

Reusing TensorBoard on port 6006 (pid 8008), started 0:03:30 ago. (Use '!kill 8008' to kill it.)

In [6]:
experiment = 'learning'

# Directory where logs will be saved. 
log_dir = os.path.join(logdir, experiment)

# initiate tensorboard summary writer
tb_writer = SummaryWriter(
    log_dir = log_dir,
    comment = "Learning How to use TensorBoard in PyTorch"
)


In [7]:
train_accuracy_tag = 'accuracy/train'
validation_accuracy_tag = 'accuracy/validation'
train_loss_tag = 'loss/train'
validation_loss_tag = 'loss/validation'

In [8]:
def dummy_accuracy_generator(iter, iter_divide_by=1):
    return 1 - np.exp(-(itr/iter_divide_by + np.random.random()))

def dummy_loss_generator(iter, iter_divide_by=1):
    return np.exp(-(itr/iter_divide_by + np.random.random()))

In [9]:
num_iteration = 50

for itr in range(num_iteration):
    tb_writer.add_scalar(tag=train_accuracy_tag, 
                         scalar_value=dummy_accuracy_generator(itr, 2),
                         global_step=itr)
    tb_writer.add_scalar(tag=validation_accuracy_tag, 
                         scalar_value=dummy_accuracy_generator(itr, 3),
                         global_step=itr)
    tb_writer.add_scalar(tag=train_loss_tag, 
                         scalar_value=dummy_loss_generator(itr, 2),
                         global_step=itr)
    tb_writer.add_scalar(tag=validation_loss_tag, 
                         scalar_value=dummy_loss_generator(itr, 3),
                         global_step=itr)

In [11]:
accuracy_tag = 'Accuracy'
loss_tag = 'Loss'

# Dictionary format to add scalars

"""
dic = {
    "Train": train loss for "Loss" tag, train accuracy for "Accuracy" tag
    "Validation": validation loss for "Loss" tag, validation accuracy for "Accuracy" tag
}
"""
for itr in range(num_iteration):
    tb_writer.add_scalars(main_tag=accuracy_tag, 
                         tag_scalar_dict={
                             "Train": dummy_accuracy_generator(itr, 2),
                             "Validation": dummy_accuracy_generator(itr, 3)
                         },
                         global_step=itr)
    tb_writer.add_scalars(main_tag=loss_tag, 
                         tag_scalar_dict={
                             "Train": dummy_loss_generator(itr, 2),
                             "Validation": dummy_loss_generator(itr, 3)
                         },
                         global_step=itr)


In [12]:
image_urls = [
    'https://farm7.staticflickr.com/6073/6032446158_85fa667cd2_z.jpg',
    'https://farm9.staticflickr.com/8538/8678472399_886f8eabec_z.jpg',
    'https://farm6.staticflickr.com/5485/10028794463_d8cbb38932_z.jpg',
    'https://farm4.staticflickr.com/3057/2475401198_0a342a907e_z.jpg'
    ]

In [13]:
# Download images.
for i in range(len(image_urls)):
    urllib.request.urlretrieve(image_urls[i], "img{}.jpg".format(i+1))

In [14]:
# Read Images.
num_images = 4

images = []
for i in range(num_images):
    img = cv2.imread('img{}.jpg'.format(i+1))
    # BGR to RGB
    img = img[...,::-1]
    images.append(img)


In [15]:
image_tag = "add_image"

for itr in range(num_images):
    tb_writer.add_image(tag=image_tag, 
                        img_tensor=images[itr],
                        global_step=itr,
                        dataformats='HWC')

In [16]:
histogram_tag = 'Image Hisograms'

for itr in range(num_images):
    x = np.random.random(1000)
    tb_writer.add_histogram(tag=histogram_tag, 
                            values=images[itr],
                            global_step=itr)

In [17]:
num_classes = 5
last_epoch = 50
sample_count = 1000

for i in range(num_classes):
    pr_curves_tag = "class {}".format(i)
    # Generate random labels
    labels = np.random.randint(2, size=sample_count)
    # Random confidence
    predictions = np.random.rand(sample_count)
    # add PR curve to tensorboard
    tb_writer.add_pr_curve(
        tag=pr_curves_tag,
        labels=labels,
        predictions=predictions,
        global_step=last_epoch
        )

In [18]:
resized_images=[]
feature_vectors = []

for img in images:
    # resize to fix size
    resized_img = cv2.resize(img, (100, 100))
    # H x W x C --> C x H x W
    resized_img = np.moveaxis(resized_img, -1, 0)
    feature_vector = np.reshape(resized_img, (-1))
    feature_vectors.append(feature_vector)
    resized_images.append(resized_img)

feature_vectors = np.array(feature_vectors)
resized_images = np.array(resized_images)
# Convert to torch tensor
resized_images = torch.from_numpy(resized_images)
feature_vectors = torch.from_numpy(feature_vectors)

print('Features size:\t{}'.format(feature_vectors.size()))
print('Images size:\t{}'.format(resized_images.size()))

Features size:	torch.Size([4, 30000])
Images size:	torch.Size([4, 3, 100, 100])


In [19]:
# label for each data point
labels = ['class 0', 'class 0', 'class 1', 'class 1']

In [20]:
embedding_tag = 'Embedding'
last_epoch = 50

tb_writer.add_embedding(mat=feature_vectors,
                        metadata=labels, 
                        label_img=resized_images, 
                        global_step=last_epoch, 
                        tag=embedding_tag
                        )

In [21]:
class MediumModel(nn.Module):
    def __init__(self):
        super().__init__()

        # convolution layers
        self._body = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5),
            nn.ReLU(inplace=True),

            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(0.2)
            )
            
        
        # Fully connected layers
        self._head = nn.Sequential(
            
            nn.Linear(in_features=64 * 22 * 22, out_features=1024), 
            nn.ReLU(inplace=True),
            nn.Dropout(0.3),
            
            nn.Linear(in_features=1024, out_features=512), 
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            
            nn.Linear(in_features=512, out_features=5)
        )

    def forward(self, x):
        x = self._body(x)
        x = x.view(x.size()[0], -1)
        x = self._head(x)
        return x

In [22]:
model = MediumModel()
# Change the image tensor datatype to float32 to make it compatible with the model.
inputs = resized_images.to(torch.float32)

In [23]:
tb_writer.add_graph(model=model, 
                    input_to_model=inputs)